In [25]:
import numpy as np

X = []
Y = []


def add_data(X, Y, csv_path, input_noises=[0.0]):
    for input_noise in input_noises:
        print(f"Adding entries with {input_noise=}")
        with open(f"../utils/datasets/{csv_path}") as f:
            for i, line in enumerate(f):
                data = line.split(",")
                x = np.array(data[2:], dtype=np.float32)
                y = np.array(data[:2], dtype=np.float32)

                y = y[1]  # steer only

                # if adding noise, redo multiple times to reduce overfitting to noise
                if input_noise > 0:
                    for cell in range(len(x)):
                        if np.random.rand() < input_noise:
                            x[cell] = 0 if x[cell] == 1 else 1  # random bit flips

                grid_size = 30
                x = x.reshape(grid_size, grid_size, 1)  # transform state back to grid

                X.append(x)
                Y.append(y)
    return X, Y


X, Y = add_data(X, Y, "kaggle_30_binary.csv", input_noises=[0.0, 0.1, 0.2])
X, Y = add_data(X, Y, "sim_oval_30_binary.csv", input_noises=[0.2, 0.3, 0.4])

X = np.array(X, dtype=np.float32)
Y = np.array(Y, dtype=np.float32)

print(f"Loaded {len(X)} entires")
print(f"entry {x.shape=}, {y.shape=}")

Adding entries with input_noise=0.0
Adding entries with input_noise=0.1
Adding entries with input_noise=0.2
Loaded 41376 entires
entry x.shape=(30, 30, 1), y.shape=()


In [26]:
import keras

model = keras.models.Sequential(
    [
        # lenet-5, modified to have double the filter count
        keras.layers.Conv2D(
            16, kernel_size=(5, 5), activation="relu", input_shape=(30, 30, 1)
        ),
        keras.layers.MaxPooling2D(pool_size=(2, 2)),
        keras.layers.Conv2D(32, kernel_size=(5, 5), activation="relu"),
        keras.layers.MaxPooling2D(pool_size=(2, 2)),
        keras.layers.Flatten(),
        keras.layers.Dense(120, activation="relu"),
        keras.layers.Dense(84, activation="relu"),
        keras.layers.Dense(1),
    ]
)

# TODO: add normal following to all experts' ds

model.compile(
    optimizer="adam",
    loss="mean_squared_error",
)

model.fit(
    X,
    Y,
    epochs=5,
    # validation_split=0.2,
    # callbacks=[
    #     keras.callbacks.EarlyStopping(patience=1),
    # ],
)

model_name = "kaggle_track_30_binary"

model_save_path = f"../shared/models/{model_name}.keras"
model.save(model_save_path)

Epoch 1/5
1293/1293 [==============================] - 8s 6ms/step - loss: 0.0157
Epoch 2/5
1293/1293 [==============================] - 8s 6ms/step - loss: 0.0121
Epoch 3/5
1293/1293 [==============================] - 8s 6ms/step - loss: 0.0106
Epoch 4/5
1293/1293 [==============================] - 8s 6ms/step - loss: 0.0092
Epoch 5/5
1293/1293 [==============================] - 9s 7ms/step - loss: 0.0081


In [22]:
import tensorflow as tf

# Convert the model
converter = tf.lite.TFLiteConverter.from_keras_model(model)
tflite_model = converter.convert()

# Save the model.
model_save_path = f"../shared/models/{model_name}.tflite"
with open(model_save_path, 'wb') as f:
  f.write(tflite_model)

INFO:tensorflow:Assets written to: /var/folders/_j/ym4tpz295j346nyxgg80y0740000gn/T/tmpphzjywzm/assets


INFO:tensorflow:Assets written to: /var/folders/_j/ym4tpz295j346nyxgg80y0740000gn/T/tmpphzjywzm/assets
2024-05-01 13:06:05.939427: W tensorflow/compiler/mlir/lite/python/tf_tfl_flatbuffer_helpers.cc:364] Ignored output_format.
2024-05-01 13:06:05.939817: W tensorflow/compiler/mlir/lite/python/tf_tfl_flatbuffer_helpers.cc:367] Ignored drop_control_dependency.
2024-05-01 13:06:05.941782: I tensorflow/cc/saved_model/reader.cc:45] Reading SavedModel from: /var/folders/_j/ym4tpz295j346nyxgg80y0740000gn/T/tmpphzjywzm
2024-05-01 13:06:05.943502: I tensorflow/cc/saved_model/reader.cc:91] Reading meta graph with tags { serve }
2024-05-01 13:06:05.943515: I tensorflow/cc/saved_model/reader.cc:132] Reading SavedModel debug info (if present) from: /var/folders/_j/ym4tpz295j346nyxgg80y0740000gn/T/tmpphzjywzm
2024-05-01 13:06:05.957537: I tensorflow/cc/saved_model/loader.cc:231] Restoring SavedModel bundle.
2024-05-01 13:06:06.097370: I tensorflow/cc/saved_model/loader.cc:215] Running initialization

In [28]:
# time inference speed
import time

start = time.time()
model(X[0:100])
end = time.time()
avg = (end - start) / 100
print(f"avg inference time: {avg*1000:.2f}ms")

avg inference time: 0.11ms
